<a href="https://colab.research.google.com/github/vboyce/AA-flowers/blob/main/code/alignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install convokit
#my github for extra pos and lemma analysis
# !pip install click


In [2]:
import convokit
from convokit import Corpus, Speaker, Utterance, download
from collections import defaultdict
from tqdm import tqdm

!python -m spacy download en_core_web_sm

2021-06-08 08:51:25.896848: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.7MB 308kB/s 
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [84]:
#data_dir = "https://raw.githubusercontent.com/georgialoukatou/Conversational-Analysis-HAI/blob/master/"
column_meta = {}
import pandas as pd

url = 'raw_chat.csv'

utts = pd.read_csv(url, encoding='utf-8')
utts = utts.loc[utts['type'] == 'message']

utts = utts.rename(columns={"gameId": "conversation_id", "targetNum": "meta", "playerId": "speaker" }) ###"XXX": "reply_to"
print(utts.index)
utts = utts.loc[utts['conversation_id'] == 'dze8j8adozpxbehCk']
#utts = utts.head(20)
#utts = utts.reset_index().rename(columns={utts.index.name:'utt_id'})
print(utts.loc[0,:])
utts = utts.T.to_dict()

print("Total number of utterances = {}".format(len(utts)))

utts[15]["conversation_id"]


Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            535, 536, 537, 538, 539, 540, 541, 542, 543, 544],
           dtype='int64', length=545)
row_id                                             2ALbdYdj59PoMKcSo
index                                                              0
stageIds                         JKxFsPHGjwBqtLR5L,tGmAqrK6qKktRtewy
conversation_id                                    dze8j8adozpxbehCk
createdAt                                       2021-05-27T01:08:24Z
context            {"purple_12":{"label":"purple_12","location":"...
meta                                                             NaN
repNum                                                             0
trialNum                                                           0
numPlayers                                                         3
text                                                           hiiii
speaker                                            vFbMfknsc7T

'dze8j8adozpxbehCk'

In [85]:
#create reply_to column 
for key in sorted(utts):
  utts[key]['reply_to'] = None
  if key==0 or utts[key]["conversation_id"] != utts[key-1]["conversation_id"] : continue
 # print(key, '->', utts[key]["speakerid"])
  i = key
  while utts[i]["speaker"] == utts[i-1]["speaker"]:
    i-=1    
  i-=1
  replier = key
  utts[key]["reply_to"] = str(replier)

for key in sorted(utts):  
  print(key, '->', utts[key]["speaker"], utts[key]["reply_to"])

0 -> vFbMfknsc7TsNLYpz None
1 -> DgHvsMothZmmKKwa6 1
2 -> DgHvsMothZmmKKwa6 2
3 -> 7HJYhZpJ86FEXBTiL 3
4 -> vFbMfknsc7TsNLYpz 4
5 -> DgHvsMothZmmKKwa6 5
6 -> vFbMfknsc7TsNLYpz 6
7 -> DgHvsMothZmmKKwa6 7
8 -> vFbMfknsc7TsNLYpz 8
9 -> 7HJYhZpJ86FEXBTiL 9
10 -> vFbMfknsc7TsNLYpz 10
11 -> vFbMfknsc7TsNLYpz 11
12 -> DgHvsMothZmmKKwa6 12
13 -> DgHvsMothZmmKKwa6 13
14 -> DgHvsMothZmmKKwa6 14
15 -> DgHvsMothZmmKKwa6 15
16 -> 7HJYhZpJ86FEXBTiL 16
17 -> 7HJYhZpJ86FEXBTiL 17
18 -> DgHvsMothZmmKKwa6 18
19 -> vFbMfknsc7TsNLYpz 19
20 -> 7HJYhZpJ86FEXBTiL 20
21 -> vFbMfknsc7TsNLYpz 21
22 -> DgHvsMothZmmKKwa6 22
23 -> DgHvsMothZmmKKwa6 23
24 -> 7HJYhZpJ86FEXBTiL 24
25 -> 7HJYhZpJ86FEXBTiL 25
26 -> vFbMfknsc7TsNLYpz 26
27 -> DgHvsMothZmmKKwa6 27
28 -> vFbMfknsc7TsNLYpz 28
29 -> DgHvsMothZmmKKwa6 29
30 -> vFbMfknsc7TsNLYpz 30
31 -> DgHvsMothZmmKKwa6 31
32 -> vFbMfknsc7TsNLYpz 32
33 -> 7HJYhZpJ86FEXBTiL 33
34 -> DgHvsMothZmmKKwa6 34
35 -> DgHvsMothZmmKKwa6 35
36 -> 7HJYhZpJ86FEXBTiL 36
37 -> vFbMfknsc7Ts

In [86]:
#convert to Convokit object

i=0
utterance_list=[]
for key in sorted(utts):
  i= i + 1
  utts[key]["utt_id"] = i
  utterance_list.append(utts[key])

utterance_list=[]
#print(help(Corpus.__init__))
#print(help(Utterance))
for key in utts:
  meta = {'target_num': utts[key]['meta']}
  tangram_utt =  Utterance(id = str(utts[key]['utt_id']), speaker=Speaker(id = str(utts[key]['speaker'])), text=str(utts[key]['text']), reply_to=(utts[key]['reply_to']), timestamp=str(utts[key]['createdAt']), meta=meta, conversation_id=str(utts[key]['conversation_id']))
  utterance_list.append(tangram_utt)

tangram_corpus = Corpus(utterances=utterance_list)

print(dir(tangram_corpus))
print(tangram_corpus.get_utterance('2'))


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_collect_speaker_data', '_merge_utterances', '_reinitialize_index_helper', '_update_corpus_speaker_data', '_vector_matrices', 'add_meta', 'add_utterances', 'append_vector_matrix', 'conversations', 'corpus_dirpath', 'delete_metadata', 'delete_vector_matrix', 'directed_pairwise_exchanges', 'dump', 'dump_info', 'dump_vectors', 'filter_conversations_by', 'filter_utterances_by', 'get_attribute_table', 'get_conversation', 'get_conversation_ids', 'get_conversations_dataframe', 'get_full_attribute_table', 'get_meta', 'get_object', 'get_object_ids', 'get_speaker', 'get_speaker_convo_attribute_table', 'get_speaker_convo_info', 'get_speaker_ids', 'get_speak

In [ ]:
#@title
#perform SpaCy analysis
#corpus = Corpus(download('subreddit-Cornell')) 

from convokit.text_processing import TextParser  
textparser = TextParser()
textparser.transform(tangram_corpus)    

In [87]:
convo = tangram_corpus.random_conversation() #show random conversation from corpus 
convo.get_utterances_dataframe()



,timestamp,text,speaker,reply_to,conversation_id,meta.target_num
id,,,,,,
1,2021-05-27T01:08:24Z,hiiii,vFbMfknsc7TsNLYpz,None,dze8j8adozpxbehCk,NaN
2,2021-05-27T01:08:24Z,hello!!,DgHvsMothZmmKKwa6,1,dze8j8adozpxbehCk,NaN
3,2021-05-27T01:08:24Z,roll call,DgHvsMothZmmKKwa6,2,dze8j8adozpxbehCk,NaN
4,2021-05-27T01:08:24Z,anyone have a full bar?,7HJYhZpJ86FEXBTiL,3,dze8j8adozpxbehCk,NaN
5,2021-05-27T01:08:24Z,not full no,vFbMfknsc7TsNLYpz,4,dze8j8adozpxbehCk,NaN
...,...,...,...,...,...,...
367,2021-05-27T01:08:24Z,orange edges?,DgHvsMothZmmKKwa6,366,dze8j8adozpxbehCk,NaN
368,2021-05-27T01:08:24Z,peckle,7HJYhZpJ86FEXBTiL,367,dze8j8adozpxbehCk,NaN
369,2021-05-27T01:08:24Z,for real this time maybe,vFbMfknsc7TsNLYpz,368,dze8j8adozpxbehCk,NaN


In [88]:
#@title
# code minimally adapted from Doyle et al, 2016 / Yurovski et al., 2016
# Hierarchical Alignment Model - model-based estimation of conditional probabilities

import operator, itertools
def group(utterances):
	utterances.sort(key=operator.itemgetter('convId'))
	list1 = []
	for key, items in itertools.groupby(utterances, operator.itemgetter('convId')):
		list1.append(list(items))
  
	return list1


def allMarkers(markers):
	categories = []
	for marker in markers:
		categories.append(marker)
	return categories

def checkMarkers(markers):
	toReturn = []
	for marker in markers:
		if isinstance(marker, str):
			toReturn.append({"marker": marker, "category": marker})
		else:
			toReturn.append(marker)
	return toReturn


def findMarkersInConvo(markers,convo):
	ba = {} # Number of times Person A and person B says the marker["marker"]
	bna = {}
	nbna = {}
	nba = {}
	for utterance in convo:				
		for j, marker in enumerate(markers):
			word = marker["marker"]
			msgMarker = word in utterance["msgMarkers"]
			replyMarker = word in utterance["replyMarkers"]
			
			if msgMarker and replyMarker:
				ba[word] = ba.get(word,0) + 1
			elif replyMarker and not msgMarker:
				bna[word] = bna.get(word,0) + 1
			elif not replyMarker and msgMarker:
				nba[word] = nba.get(word,0) + 1
			else:
				nbna[word] = nbna.get(word,0) + 1
	#		print(msgMarker, replyMarker)
			print(ba, bna, nba, nbna)
	return({'ba': ba,'bna': bna,'nba': nba,'nbna': nbna}) 
 

def addFeats(toAppend,utterance,renameIds=True,corpusType=''):
	if renameIds:
		toAppend["speakerId"] = utterance["speakerId"]
		toAppend["replierId"] = utterance["replierId"]
	else:
		toAppend["speakerId"] = utterance["speakerId"]
		toAppend["replierId"] = utterance["replierId"]		
	#if(corpusType=='Twitter'):
	#	toAppend["reciprocity"] = utterance["reciprocity"]
	#	toAppend["verifiedSpeaker"] = bool(utterance["verifiedSpeaker"])
	#	toAppend["verifiedReplier"] = bool(utterance["verifiedReplier"])
	#	toAppend["speakerFollowers"] = utterance["speakerFollowers"]
	#	toAppend["replierFollowers"] = utterance["replierFollowers"]			
	#elif(corpusType=='CHILDES'):
	#	toAppend["corpus"] = utterance["corpus"]
	#	toAppend["docId"] = utterance["docId"]
	return(toAppend)


def metaDataExtractor(groupedUtterances, markers,corpusType=''):
	results = []
	for i, convo in enumerate(groupedUtterances):
		toAppend = findMarkersInConvo(markers,convo)
		toAppend = addFeats(toAppend,convo[0],True,corpusType)		
		results.append(toAppend)
	return results

def readMarkers(markersFile,dialect=None):
	if dialect is None:
		reader = csv.reader(open(markersFile))
	else:
		reader = csv.reader(open(markersFile),dialect=dialect)
	markers = []
	print('marker\tcategory')
	for i, row in enumerate(reader):
		toAppend = {}
		toAppend["marker"] = row[0]
		if(len(row) > 1):
			toAppend["category"] = row[1]
		else:
			toAppend["category"] = row[0]
		markers.append(toAppend)
		#print(toAppend["marker"]+'\t'+toAppend["category"])
	return markers

def writeFile(results, outputFile, shouldWriteHeader):
	if len(results) == 0:
		print("x")
		return
	toWrite = []
	header = sorted(list(results[0].keys()))
	for row in results:
		toAppend = []
		for key in header:
			toAppend.append(row[key])
		toWrite.append(toAppend)
	if shouldWriteHeader:
		with open(outputFile, "w", newline='') as f:
			writer = csv.writer(f)
			writer.writerows([header])
		f.close()
	with open(outputFile, "a", newline='') as f:
		writer = csv.writer(f)
		writer.writerows(toWrite)
	f.close()

def determineCategories(msgMarkers,catdict,useREs=False):
	msgCats = []
	#iterate over catdict items {category: [words/REs]}
	for cd in catdict.items():
		if useREs:
			if any(any(wordre.match(marker) for marker in msgMarkers) for wordre in cd[1]):	#if REs, see if any tokens match each RE
				msgCats.append(cd[0])
		else:
			if any(word in msgMarkers for word in cd[1]):			#if just words, see if any word in category also in msg
				msgCats.append(cd[0])
	return msgCats

def makeCatDict(markers,useREs=False):
	mdict = {}
	for m in markers:
		marker = re.compile(''.join([m["marker"], '$'])) if useREs else m["marker"]
		if m["category"] in mdict:
			mdict[m["category"]].append(marker)
		else:
			mdict[m["category"]] = [marker]
		#mdict[m["category"]] = mdict.get(m["category"],[]).append(m["marker"])	#Need to swap marker and category labels
		#mdict[m["marker"]] = mdict.get(m["marker"],[]).append(m["category"])
	return(mdict)
 
#important for alignment: 
## not affected by frequency differences of different items --> estimate different alignment values for different markers
## provides direction of linguistic similarity
## controls for a priori similarity (homophily)
## sparse data




def createAlignmentDict(category,result,smoothing,corpusType=''):
	toAppend = {}
	print(category)
	print("R", result)
	ba = int(result["ba"].get(category, 0))
	bna = int(result["bna"].get(category, 0))
	nbna = int(result["nbna"].get(category, 0))
	nba = int(result["nba"].get(category, 0))
	print(ba,bna,nbna,nba)
	#Calculating alignment only makes sense if we've seen messages with and without the marker
	if (((ba+nba)==0 or (bna+nbna)==0)):
		return(None)
  
	toAppend = addFeats(toAppend,result,False,corpusType)
	toAppend["category"] = category
		
	#Calculating Echoes of Power alignment 
	powerNum = ba
	powerDenom = ba+nba
	baseNum = ba+bna
	baseDenom = ba+nba+bna+nbna

	if(powerDenom != 0 and baseDenom != 0):
		dnmalignment = powerNum/powerDenom - baseNum/baseDenom
		toAppend["dnmalignment"] = dnmalignment
	else:
		toAppend["dnmalignment"] = False
	
	powerNum = ba
	powerDenom = ba+nba
	baseDenom = bna+nbna
	baseNum = bna
	powerProb = math.log((powerNum+smoothing)/float(powerDenom+2*smoothing))
	baseProb = math.log((baseNum+smoothing)/float(baseDenom+2*smoothing))
	alignment = powerProb - baseProb #alignment / not b - not a / not a + not a not b
	toAppend["alignment"] = alignment
	
	toAppend["ba"] = ba
	toAppend["bna"] = bna
	toAppend["nba"] = nba
	toAppend["nbna"] = nbna
	return(toAppend)
 

In [89]:
def runFormula(results, markers, smoothing,corpusType):
  toReturn = []
  categories = allMarkers(markers)
  for i, result in enumerate(results):
    for j, category in enumerate(categories):
      toAppend = createAlignmentDict(category["category"],result,smoothing,corpusType)
      if toAppend is not None:
        toReturn.append(toAppend)
  print(toReturn)

  toReturn = sorted(toReturn, key=lambda k: (k["replierId"],k["speakerId"],k["category"]))
  #	toReturn = sorted(toReturn)

  return toReturn



In [90]:
#@title
convo_utts= list(convo.iter_utterances()) 
convo_utts


[Utterance({'obj_type': 'utterance', 'meta': {'target_num': nan}, 'vectors': [], 'speaker': Speaker({'obj_type': 'speaker', 'meta': {}, 'vectors': [], 'owner': None, 'id': 'vFbMfknsc7TsNLYpz'}), 'conversation_id': 'dze8j8adozpxbehCk', 'reply_to': None, 'timestamp': '2021-05-27T01:08:24Z', 'text': 'hiiii', 'owner': <convokit.model.corpus.Corpus object at 0x7f79eca441d0>, 'id': '1'}),
 Utterance({'obj_type': 'utterance', 'meta': {'target_num': nan}, 'vectors': [], 'speaker': Speaker({'obj_type': 'speaker', 'meta': {}, 'vectors': [], 'owner': None, 'id': 'DgHvsMothZmmKKwa6'}), 'conversation_id': 'dze8j8adozpxbehCk', 'reply_to': '1', 'timestamp': '2021-05-27T01:08:24Z', 'text': 'hello!!', 'owner': <convokit.model.corpus.Corpus object at 0x7f79eca441d0>, 'id': '2'}),
 Utterance({'obj_type': 'utterance', 'meta': {'target_num': nan}, 'vectors': [], 'speaker': Speaker({'obj_type': 'speaker', 'meta': {}, 'vectors': [], 'owner': None, 'id': 'DgHvsMothZmmKKwa6'}), 'conversation_id': 'dze8j8adozpx

In [91]:
import pprint

# pair utterances from convokit 
markers = ["the", "of"]
#"very", "to", "through", "and","each", "other", "'d", "more", "less", "front", "back", "please", "any", "due", "just", "a", "in", "that", "for", "on", "are", "you", "it", "was", "were", "I", "as", "with", "they", "be", "at", "too", "have", "does", "this", "from", "or", "had", "by", "but", "some", "also", "what", "can", "out", "all", "your", "my", "up", "even", "so", "yes", "when", "almost", "no", "must", "should", "will", "would", "not", "a", "that", "at"

utterances =[]
previous_utt = dict()
#convo = corpus.random_conversation()


for i in convo.traverse('bfs'): #iterate through utterances in tree-like (breadth) structure, level by level
  #print(i)
  d = {} #each pair of speakers is a 'd' dictionary
  previous_utt[i.id] = (i.text,i.speaker.id)
 #print(previous_utt)
  if i.reply_to == None:  #ignore if no reply
    continue
  d['convId'] = (previous_utt[i.reply_to][1], i.speaker.id)
 # print(d['convId'])
  d['speakerId'] = previous_utt[i.reply_to][1] #?
 # print(d['speakerId'])
  d['msg'] = previous_utt[i.reply_to][0]
 # print(d['msg'])
  d['reply'] = i.text
 # print(d['reply'])
  d['msgMarkers'] = []
  #print(d['msgMarkers'])
  d['replyMarkers'] = []
  d['msgTokens']= []
  #print(d['msgTokens'])
  d['replyTokens']= []
  d["replierId"] = i.speaker.id
 # print(d['replierId'])
 # pprint.pprint(d)
  utterances.append(d)



In [56]:
for i in convo.iter_utterances():
  print(i)

Utterance(id: '1', conversation_id: dze8j8adozpxbehCk, reply-to: None, speaker: Speaker(id: vFbMfknsc7TsNLYpz, vectors: [], meta: {}), timestamp: 2021-05-27T01:08:24Z, text: 'hiiii', vectors: [], meta: {'target_num': nan})
Utterance(id: '2', conversation_id: dze8j8adozpxbehCk, reply-to: 1, speaker: Speaker(id: DgHvsMothZmmKKwa6, vectors: [], meta: {}), timestamp: 2021-05-27T01:08:24Z, text: 'hello!!', vectors: [], meta: {'target_num': nan})
Utterance(id: '3', conversation_id: dze8j8adozpxbehCk, reply-to: 2, speaker: Speaker(id: 7HJYhZpJ86FEXBTiL, vectors: [], meta: {}), timestamp: 2021-05-27T01:08:24Z, text: 'anyone have a full bar?', vectors: [], meta: {'target_num': nan})
Utterance(id: '4', conversation_id: dze8j8adozpxbehCk, reply-to: 3, speaker: Speaker(id: vFbMfknsc7TsNLYpz, vectors: [], meta: {}), timestamp: 2021-05-27T01:08:24Z, text: 'not full no', vectors: [], meta: {'target_num': nan})
Utterance(id: '5', conversation_id: dze8j8adozpxbehCk, reply-to: 4, speaker: Speaker(id: Dg

In [92]:
for idx, utt in enumerate(utterances):  #append marker in utterance metadata if it exists
  for marker in markers:
			if marker in utt['msg']:
				utterances[idx]['msgMarkers'].append(marker)
			if marker in utt['reply']:
				utterances[idx]['replyMarkers'].append(marker)

pprint.pprint(utterances)

[{'convId': ('vFbMfknsc7TsNLYpz', 'DgHvsMothZmmKKwa6'),
  'msg': 'hiiii',
  'msgMarkers': [],
  'msgTokens': [],
  'replierId': 'DgHvsMothZmmKKwa6',
  'reply': 'hello!!',
  'replyMarkers': [],
  'replyTokens': [],
  'speakerId': 'vFbMfknsc7TsNLYpz'},
 {'convId': ('DgHvsMothZmmKKwa6', 'DgHvsMothZmmKKwa6'),
  'msg': 'hello!!',
  'msgMarkers': [],
  'msgTokens': [],
  'replierId': 'DgHvsMothZmmKKwa6',
  'reply': 'roll call',
  'replyMarkers': [],
  'replyTokens': [],
  'speakerId': 'DgHvsMothZmmKKwa6'},
 {'convId': ('DgHvsMothZmmKKwa6', '7HJYhZpJ86FEXBTiL'),
  'msg': 'roll call',
  'msgMarkers': [],
  'msgTokens': [],
  'replierId': '7HJYhZpJ86FEXBTiL',
  'reply': 'anyone have a full bar?',
  'replyMarkers': [],
  'replyTokens': [],
  'speakerId': 'DgHvsMothZmmKKwa6'},
 {'convId': ('7HJYhZpJ86FEXBTiL', 'vFbMfknsc7TsNLYpz'),
  'msg': 'anyone have a full bar?',
  'msgMarkers': [],
  'msgTokens': [],
  'replierId': 'vFbMfknsc7TsNLYpz',
  'reply': 'not full no',
  'replyMarkers': [],
  'reply

In [93]:
#main function - run alignment
import math, csv

smoothing=1
shouldWriteHeader=True
outputFile='output.csv'
markers = checkMarkers(markers)
groupedUtterances = group(utterances)
metaData = metaDataExtractor(groupedUtterances,markers) 
print(metaData)        


{} {} {} {'the': 1}
{} {} {} {'the': 1, 'of': 1}
{} {} {'the': 1} {'the': 1, 'of': 1}
{} {} {'the': 1} {'the': 1, 'of': 2}
{} {} {'the': 1} {'the': 2, 'of': 2}
{} {} {'the': 1} {'the': 2, 'of': 3}
{} {} {'the': 1} {'the': 3, 'of': 3}
{} {} {'the': 1} {'the': 3, 'of': 4}
{} {} {'the': 1} {'the': 4, 'of': 4}
{} {} {'the': 1} {'the': 4, 'of': 5}
{} {} {'the': 1} {'the': 5, 'of': 5}
{} {} {'the': 1} {'the': 5, 'of': 6}
{} {} {'the': 1} {'the': 6, 'of': 6}
{} {} {'the': 1} {'the': 6, 'of': 7}
{} {} {'the': 1} {'the': 7, 'of': 7}
{} {} {'the': 1} {'the': 7, 'of': 8}
{} {'the': 1} {'the': 1} {'the': 7, 'of': 8}
{} {'the': 1} {'the': 1} {'the': 7, 'of': 9}
{} {'the': 1} {'the': 1} {'the': 8, 'of': 9}
{} {'the': 1} {'the': 1} {'the': 8, 'of': 10}
{} {'the': 1} {'the': 1} {'the': 9, 'of': 10}
{} {'the': 1} {'the': 1} {'the': 9, 'of': 11}
{} {'the': 1} {'the': 1} {'the': 10, 'of': 11}
{} {'the': 1} {'the': 1} {'the': 10, 'of': 12}
{} {'the': 1} {'the': 1} {'the': 11, 'of': 12}
{} {'the': 1} {'the

In [94]:

results = runFormula(metaData, markers, smoothing,'')


the
R {'ba': {}, 'bna': {'the': 1}, 'nba': {'the': 1}, 'nbna': {'the': 20, 'of': 22}, 'speakerId': '7HJYhZpJ86FEXBTiL', 'replierId': '7HJYhZpJ86FEXBTiL'}
0 1 20 1
of
R {'ba': {}, 'bna': {'the': 1}, 'nba': {'the': 1}, 'nbna': {'the': 20, 'of': 22}, 'speakerId': '7HJYhZpJ86FEXBTiL', 'replierId': '7HJYhZpJ86FEXBTiL'}
0 0 22 0
the
R {'ba': {'the': 1}, 'bna': {'the': 11, 'of': 2}, 'nba': {'the': 2, 'of': 1}, 'nbna': {'the': 37, 'of': 48}, 'speakerId': '7HJYhZpJ86FEXBTiL', 'replierId': 'DgHvsMothZmmKKwa6'}
1 11 37 2
of
R {'ba': {'the': 1}, 'bna': {'the': 11, 'of': 2}, 'nba': {'the': 2, 'of': 1}, 'nbna': {'the': 37, 'of': 48}, 'speakerId': '7HJYhZpJ86FEXBTiL', 'replierId': 'DgHvsMothZmmKKwa6'}
0 2 48 1
the
R {'ba': {'the': 1}, 'bna': {'the': 7}, 'nba': {'the': 2, 'of': 1}, 'nbna': {'the': 35, 'of': 44}, 'speakerId': '7HJYhZpJ86FEXBTiL', 'replierId': 'vFbMfknsc7TsNLYpz'}
1 7 35 2
of
R {'ba': {'the': 1}, 'bna': {'the': 7}, 'nba': {'the': 2, 'of': 1}, 'nbna': {'the': 35, 'of': 44}, 'speakerId': 

In [96]:
#print(results)

writeFile(results, outputFile, shouldWriteHeader)


In [180]:
#@title
#check use of PoS and word length
import collections

def word_length(convo):
  d = collections.defaultdict(list)
  for spkr in convo.iter_speakers():
    spkr_utts = list(spkr.iter_utterances())
    #print(spkr_utts)
    for i in spkr_utts:
        d[spkr.id].append(len(i.text.split(" "))) #counts number of words #FIX
  return(d)



def pos(convo):
    d2 = collections.defaultdict(list)
    for spkr in convo.iter_speakers():
      spkr_utts = list(spkr.iter_utterances())
      #print(spkr_utts)
      for i in spkr_utts:
        for j in i.meta["parsed"]: 
          d2[spkr.id].append(list())
          for k in range(len(j["toks"])) :
            d2[spkr.id][-1].append(j["toks"][k]["pos"]) #find "parsed" pos metadata for each speaker (separate by utt)
    return(d2)


#word_length(convo)
#pos(convo)

#d2 gives parts of speech for each utterance and each speaker)
